In [1]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 2.7 MB/s 
     |████████████████████████████████| 662 kB 36.7 MB/s 
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 88 kB 3.5 MB/s 
     |████████████████████████████████| 5.2 MB 13.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 5.8 MB 39.5 MB/s 
     |████████████████████████████████| 1.3 MB 59.0 MB/s 
     |████████████████████████████████| 182 kB 59.1 MB/s 
     |████████████████████████████████| 7.6 MB 29.5 MB/s 
     |████████████████████████████████| 1.1 MB 59.4 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700839 sha256=c7d55cd78efceaded1e3c45ec92690ad6be6148d29bec2ea7163b39da747a649
  Stored in directory: /root/.cache/

In [2]:
!pip install konlpy pecab gensim
!curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 26.4 MB 1.3 MB/s 
     |████████████████████████████████| 465 kB 47.5 MB/s 
     |████████████████████████████████| 131 kB 42.5 MB/s 
  Created wheel for pecab: filename=pecab-1.0.2-py3-none-any.whl size=26646461 sha256=c345829eb806a7531be563ff310b8049c15be998c9d30fa8ddbcb1b74890d9a0
  Stored in directory: /root/.cache/pip/wheels/57/e2/87/fe894d7193d71e94506f6bb86affe21bdfdebe8f6f2b665a1c
Successfully built pecab
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_au

In [3]:
import pandas as pd
from tqdm.notebook import tqdm
# 형태소 분석기 불러오기
from konlpy.tag import Mecab; mecab = Mecab()
# CountVectorizer 불러오기 -> document-term matrix를 만들어주는 함수.
from sklearn.feature_extraction.text import CountVectorizer
# bertopic 불러오기
from bertopic import BERTopic

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# file_name = input("텍스트 파일 이름을 입력하세요 : ")
# start_date = input("시작 날짜를 입력하세요 : ")
# end_date = input("마지막 날짜를 입력하세요 : ")

# Colab Notebooks
base_path = "/content/drive/MyDrive/Colab Notebooks/DScourse/week15/"

# 아까 크롤링한 파일
## 크롤링한 파일이 바뀌면, 경로도 수정되어야 합니다.
# data_path = f"{file_name}_({start_date}~{end_date}).xlsx"
data_path = '월드컵_(2022.10.01~2022.12.04).xlsx'

In [8]:
data = pd.read_excel(base_path + data_path)

contents = data['contents'].dropna()
contents

0       '12년 만의 16강' 한국, 호화멤버 브라질 상대로 6일 '원정 8강 새역사' 도...
1       2022 카타르 월드컵 조별리그 H조 3차전 대한민국과 포르투갈 경기에서 권경원이 ...
2       기사내용 요약황희찬, 교체 26분 만에 골…안정환은 27분 걸려첫 한 대회 2경기 ...
3       네덜란드·칠레도 월드컵 본선서 세계 1위와 세 차례 만나브라질, 월드컵 16강전 패...
4       3일 오전(한국시간) 카타르 알라이얀의 에듀케이션 시티 스타디움에서 열린 2022 ...
                              ...                        
1563    A조에서는 네덜란드와 세네갈이 16강 진출…카타르는 3경기 전패    세네갈 쿨리발...
1564    독일, 조별리그 2차전까지 1무1패로 최하위코스타리카전 무조건 이겨야 16강행 가능...
1565    치킨[연합뉴스TV 제공]    (서울=연합뉴스) 신선미 기자 = 한국 축구대표팀의 ...
1566    16강 진출 확정한 브라질, 12월 2일 카메룬과 조별리그 최종전    세르비아전 ...
1567    김민재 '끝까지 쫓아간다'(알라이얀=연합뉴스) 임헌정 기자 = 24일 오후(현지시간...
Name: contents, Length: 1565, dtype: object

In [9]:
p_contents = []
for doc in tqdm(contents):
    tokens = [token for token in mecab.morphs(doc) if len(token) > 1]
    p_contents.append(" ".join(tokens))

print(p_contents[0])

  0%|          | 0/1565 [00:00<?, ?it/s]

12 16 한국 호화 멤버 브라질 상대 원정 역사 도전 카타르 월드컵 16 진출 태극 전사 알라 연합뉴스 임헌정 기자 오전 한국 시간 카타르 알라 에듀케이션 시티 스타디움 에서 열린 2022 카타르 월드컵 조별 리그 대한민국 포르투갈 경기 에서 승리 거두 16 진출 성공 대표 기념 촬영 기쁨 나누 2022 12 kane yna co kr 도하 연합뉴스 진남 기자 12 월드컵 16 진출 이룬 한국 축구 이제 사상 원정 대회 라는 새로운 도전 나선다 그런데 상대 세계 최강 브라질 파울루 벤투 감독 이끄 한국 축구 대표 오전 이하 한국 시간 카타르 도하 스타디움 974 에서 브라질 2022 국제축구연맹 FIFA 카타르 월드컵 16 강전 치른다 한국 포르투갈 조별 리그 마지막 에서 후반 46 터진 황희 울버햄프턴 결승 역전승 거두 16 진출 나란히 승점 기록 우루과이 득실 까지 지만 다득점 에서 앞선 우리나라 포르투갈 16 오르 기적 만들 우리 나라 월드컵 에서 16 오른 2010 남아프리카 공화국 대회 이후 12 신화 2002 대회 포함 통산 이제 원정 월드컵 16 라는 역사 쓰려 지만 상대 너무 차지 16 진출 브라질 올해 22 월드컵 에서 빠짐없이 본선 참가 통산 최다 이나 우승 차지 축구 강국 현재 FIFA 랭킹 한국 28 카타르 에서 강력 우승 후보 꼽힌다 돌파 주니오르 루사일 연합뉴스 김주성 기자 24 오후 현지 시간 카타르 루사일 스타디움 에서 열린 2022 카타르 월드컵 조별 리그 브라질 세르비아 경기 브라질 비니시우스 주니오르 세르비아 파블로비치 다툼 벌이 2022 11 25 utzza yna co kr 선수 면면 화려 최종 명단 26 22 유럽 프로 축구 빅리그 잉글랜드 스페인 프랑스 이탈리아 에서 몸값 어마어마 선수 시장 가치 분석 축구 사이트 트랜스퍼 마르크 따르 이번 대회 참가 브라질 선수 26 시장 가치 적료 추정 총액 11 천만 유로 600 이번 대회 본선 참가 32 개국 잉글랜드 12 천만 유로 브라질 에서 가장 시장 가치 선수 측면 공격

Source : https://www.sbert.net/docs/pretrained_models.html

In [10]:
vectorizer = CountVectorizer(max_features=3000) # 빈도가 높은 3000개만 사용.

# sentence-transformer에 있는 pretrained model 사용 가능.
model = BERTopic(embedding_model='paraphrase-multilingual-mpnet-base-v2', # pretrained-model
                 vectorizer_model=vectorizer, # vectorizer
                 nr_topics=10,  # num_topics
                 top_n_words=10, # top_n_words
                 calculate_probabilities=True)

In [11]:
topics, probs = model.fit_transform(p_contents) # dtm

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [12]:
# Intertopic Distance Map
model.visualize_topics()

In [13]:
# Topic Probability Distribution
model.visualize_distribution(probs[0], min_probability=0.0001)

In [14]:
# Show top words
model.visualize_barchart(top_n_topics=10, n_words=10)

In [15]:
# Print each topic info
for i in range(10):
    print(f'{i}번째 토픽 : {model.get_topic(i)}')

0번째 토픽 : [('에서', 0.04052733861336144), ('한국', 0.03630381641710296), ('포르투갈', 0.03511164894581312), ('경기', 0.03444879861381898), ('손흥민', 0.03272054769469009), ('응원', 0.03257375637769899), ('16', 0.032125081064463004), ('월드컵', 0.030228994415593267), ('대표', 0.02987044235691745), ('선수', 0.02962876962235269)]
1번째 토픽 : [('한국', 0.04319956532638518), ('에서', 0.042959048408127073), ('포르투갈', 0.037562357649419856), ('월드컵', 0.034920409271879546), ('카타르', 0.0328376052243941), ('경기', 0.032560128779398255), ('16', 0.030742443862499764), ('알라', 0.0294939260779228), ('2022', 0.029326934395700534), ('으로', 0.028489414589081014)]
2번째 토픽 : [('브라질', 0.060535964295418056), ('훈련', 0.058635977898504756), ('네이마르', 0.04820347708659776), ('에서', 0.04011147476161886), ('김민재', 0.03690570364006792), ('출전', 0.0365801357156326), ('부상', 0.03432122897274258), ('카타르', 0.032878012724825746), ('한국', 0.0324145120985377), ('대표', 0.028427708185134116)]
3번째 토픽 : [('일본', 0.1239968290112171), ('스페인', 0.07778197519605778), ('독일', 0

#### References

1. Bertopic documentation : https://maartengr.github.io/BERTopic/index.html

2. Bertopic github : https://github.com/MaartenGr/BERTopic

3. 한국어 버토픽(Korean BERTopic) : https://wikidocs.net/162079